<a href="https://colab.research.google.com/github/saurabhmungale/DataScience_Assignements/blob/main/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Que.1Implement a simple GAN architecture to generate random images (like noise or basic shapes) using
TensorFlow/KerasB

In [ ]:
def sample_images(epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, noise_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)

    # If only one row, axs is 1D; otherwise, it's 2D.
    count = 0
    for i in range(dim[0]):
        for j in range(dim[1]):
            # If axs is a 2D array, axs[i, j] will work. Otherwise, axs is a 1D array
            if dim[0] == 1:  # if we only have one row of images
                axs[j].imshow(generated_images[count, :, :, 0], cmap='gray')
                axs[j].axis('off')
            else:
                axs[i, j].imshow(generated_images[count, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
            count += 1

    plt.savefig(f"gan_generated_image_{epoch}.png")
    plt.close()


Que.2.Implement the discriminator for a GAN with an image input of shape (28, 28)B

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the shape of the image input (28x28 grayscale images)
img_shape = (28, 28, 1)

# Build the Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential()

    # First convolutional layer
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))  # LeakyReLU activation
    model.add(layers.Dropout(0.25))  # Dropout for regularization

    # Second convolutional layer
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # Third convolutional layer
    model.add(layers.Conv2D(256, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # Flatten the output
    model.add(layers.Flatten())

    # Fully connected layer with sigmoid activation
    model.add(layers.Dense(1, activation='sigmoid'))  # Output a value between 0 and 1 (real/fake)

    return model

# Create the Discriminator
discriminator = build_discriminator()

# Compile the Discriminator Model
discriminator.compile(
    loss='binary_crossentropy',  # Binary crossentropy for classification
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),  # Adam optimizer
    metrics=['accuracy']  # Track accuracy during training
)

# Print the Discriminator Model summary to check the architecture
discriminator.summary()


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)                   │ (None, 14, 14, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_28 (LeakyReLU)           │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_29 (LeakyReLU)           │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 4, 4, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_30 (LeakyReLU)           │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │           4,097 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 373,761 (1.43 MB)

 Trainable params: 373,761 (1.43 MB)

 Non-trainable params: 0 (0.00 B)

QUe.3.Train the generator to produce simple digits (using noise as input) and plot the generated imagesB

In [ ]:
# Build the Generator (updated)
def build_generator():
    model = tf.keras.Sequential()

    # Fully connected layer
    model.add(layers.Dense(7 * 7 * 128, input_dim=noise_dim))  # 7*7*128 = 3136
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))

    # Reshape to 7x7x128
    model.add(layers.Reshape((7, 7, 128)))

    # Upsampling layers
    model.add(layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))  # 14x14x128
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))

    model.add(layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))  # 28x28x64
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))

    model.add(layers.Conv2DTranspose(1, kernel_size=3, strides=1, padding='same', activation='tanh'))  # 28x28x1

    return model


QUe.4. Implement WGAN by modifying the loss function in the GANB

In [22]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Define the dimensions of the random noise vector
noise_dim = 100

# Define the shape of the generated images (e.g., 28x28 for MNIST-like images)
img_shape = (28, 28, 1)

# Build the Generator Model
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=noise_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(np.prod(img_shape), activation='tanh'))
    model.add(layers.Reshape(img_shape))
    return model

# Build the Critic (Discriminator) Model
def build_critic():
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1))  # No activation function for the critic
    return model

# Compile the Critic Model
critic = build_critic()
critic.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005))

# Build the WGAN (stacked generator and critic)
def build_wgan(generator, critic):
    model = tf.keras.Sequential()
    model.add(generator)
    critic.trainable = False
    model.add(critic)
    return model

# Create instances of generator and WGAN models
generator = build_generator()
wgan = build_wgan(generator, critic)

# Compile the WGAN model
wgan.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005))

# Function to train the WGAN
def train_wgan(epochs, batch_size, sample_interval=50):
    # Load the MNIST dataset (we'll use this as our "real" dataset)
    (X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    X_train = X_train / 127.5 - 1.0  # Rescale to [-1, 1]
    X_train = np.expand_dims(X_train, axis=3)  # Add channel dimension

    # Labels for real and fake images (not used in WGAN)
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train the Critic

        # Train the critic 5 times per generator update
        for _ in range(5):  # Number of critic updates
            # Select a random batch of real images from the dataset
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_images = X_train[idx]

            # Generate fake images using the generator
            noise = np.random.normal(0, 1, (batch_size, noise_dim))
            fake_images = generator.predict(noise)

            # Train the critic on real and fake images
            c_loss_real = critic.train_on_batch(real_images, real_labels)
            c_loss_fake = critic.train_on_batch(fake_images, fake_labels)
            c_loss = 0.5 * np.add(c_loss_real, c_loss_fake)

            # Clip the critic's weights
            for layer in critic.layers:
                if hasattr(layer, 'kernel'):
                    layer.kernel.assign(tf.clip_by_value(layer.kernel, -0.01, 0.01))

        # Train the Generator
        noise = np.random.normal(0, 1, (batch_size, noise_dim))
        g_loss = wgan.train_on_batch(noise, real_labels)  # Generator tries to fool the critic

        # Print the progress
        if epoch % sample_interval == 0:
            print(f"{epoch} [C loss: {c_loss}] [G loss: {g_loss}]")
            sample_images(epoch)

# Function to save generated images
def sample_images(epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, noise_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    # Check the shape of generated_images before plotting
    print(generated_images.shape)  # Debugging print to check shape

    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)
    count = 0

    # If there's only one row of images, axs will be a 1D array, not 2D.
    # So, we need to ensure axs is always 2D by checking its shape.
    if dim[0] == 1:
        axs = np.expand_dims(axs, axis=0)

    for i in range(dim[0]):
        for j in range(dim[1]):
            # Make sure to handle the correct number of images
            axs[i, j].imshow(generated_images[count, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            count += 1
    plt.savefig(f"wgan_generated_image_{epoch}.png")
    plt.close()



Que.5.B Use a trained generator to generate a batch of fake images and display them

In [20]:
# Function to save generated images
def sample_images(epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, noise_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    # Check the shape of generated_images before plotting
    print(generated_images.shape)  # Debugging print to check shape

    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)
    count = 0
    for i in range(dim[0]):
        for j in range(dim[1]):
            axs[i, j].imshow(generated_images[count, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            count += 1
    plt.savefig(f"wgan_generated_image_{epoch}.png")
    plt.close()


Que.6.Create a StyleGAN-inspired architecture that outputs high-resolution imagesB

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Latent vector size (input to the network)
latent_dim = 512
# Image shape (output resolution)
img_shape = (1024, 1024, 3)

# Define the Mapping Network
def build_mapping_network():
    model = tf.keras.Sequential()
    model.add(layers.InputLayer(input_shape=(latent_dim,)))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))  # Style vector w
    return model

# Define the StyleGAN Generator Network
def build_generator():
    # Mapping network for style vector
    style_input = layers.Input(shape=(latent_dim,))
    style_network = build_mapping_network()
    w = style_network(style_input)

    # Synthesis network
    x = layers.Dense(4*4*512)(w)
    x = layers.Reshape((4, 4, 512))(x)
    x = layers.UpSampling2D()(x)  # Upsampling to 8x8
    x = layers.Conv2D(512, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    # Progressively upscale
    x = layers.UpSampling2D()(x)  # Upsampling to 16x16
    x = layers.Conv2D(256, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 32x32
    x = layers.Conv2D(128, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 64x64
    x = layers.Conv2D(64, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 128x128
    x = layers.Conv2D(32, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 256x256
    x = layers.Conv2D(16, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 512x512
    x = layers.Conv2D(8, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 1024x1024
    x = layers.Conv2D(4, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    # Final output layer
    img = layers.Conv2D(3, kernel_size=3, padding='same', activation='tanh')(x)

    # Create and return the model
    model = tf.keras.Model(inputs=style_input, outputs=img)
    return model

# Define the Discriminator
def build_discriminator():
    input_img = layers.Input(shape=img_shape)

    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(input_img)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1, activation='sigmoid')(x)  # Binary output (real or fake)

    model = tf.keras.Model(inputs=input_img, outputs=x)
    return model

# Instantiate the models
generator = build_generator()
discriminator = build_discriminator()

# Compile the discriminator model
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

# Build the GAN model (combine generator and discriminator)
discriminator.trainable = False  # During GAN training, freeze the discriminator

z = layers.Input(shape=(latent_dim,))
gen_img = generator(z)
validity = discriminator(gen_img)

wgan = tf.keras.Model(inputs=z, outputs=validity)
wgan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Function to save generated images
def save_generated_images(epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)
    count = 0
    for i in range(dim[0]):
        for j in range(dim[1]):
            axs[i, j].imshow(generated_images[count])
            axs[i, j].axis('off')
            count += 1
    plt.savefig(f"generated_image_epoch_{epoch}.png")
    plt.close()



# Function to train the GAN model
def train_gan(epochs, batch_size, sample_interval=50):
    # Load dataset (e.g., CIFAR-10, here we use random data for illustration)
   # (X_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()
    #X_train = X_train / 127.5 - 1.0  # Rescale to [-1, 1]
    #X_train = np.expand_dims(X_train, axis=3)  # Add channel dimension
    from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

    # Resize CIFAR-10 images to 1024x1024
    # Correct the indentation of this block
    (X_train, _), (_, _) = tf.keras.datasets.cifar10.load_data() # load the cifar10 data
    X_train_resized = np.array([np.array(array_to_img(img)).resize((1024, 1024)) for img in X_train])
    X_train_resized = X_train_resized / 127.5 - 1.0  # Rescale to [-1, 1]


    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train Discriminator

        # Select a random batch of real images
        idx = np.random.randint(0, X_train_resized.shape[0], batch_size) # use X_train_resized instead of X_train
        real_images = X_train_resized[idx]

        # Generate fake images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_images = generator.predict(noise)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator (via the GAN model)
        g_loss = wgan.train_on_batch(noise, real_labels)

        # Print the progress
        if epoch % sample_interval == 0:
            print(f"{epoch} [D loss: {d_loss}] [G loss: {g_loss}]")
            save_generated_images(epoch)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Que.7.Implement the Wasserstein loss function for GAN trainingB

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Latent vector size (input to the network)
latent_dim = 512
# Image shape (output resolution)
img_shape = (1024, 1024, 3)

# Define the Mapping Network
def build_mapping_network():
    model = tf.keras.Sequential()
    model.add(layers.InputLayer(input_shape=(latent_dim,)))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))  # Style vector w
    return model

# Define the StyleGAN Generator Network
def build_generator():
    # Mapping network for style vector
    style_input = layers.Input(shape=(latent_dim,))
    style_network = build_mapping_network()
    w = style_network(style_input)

    # Synthesis network
    x = layers.Dense(4*4*512)(w)
    x = layers.Reshape((4, 4, 512))(x)
    x = layers.UpSampling2D()(x)  # Upsampling to 8x8
    x = layers.Conv2D(512, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    # Progressively upscale
    x = layers.UpSampling2D()(x)  # Upsampling to 16x16
    x = layers.Conv2D(256, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 32x32
    x = layers.Conv2D(128, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 64x64
    x = layers.Conv2D(64, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 128x128
    x = layers.Conv2D(32, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 256x256
    x = layers.Conv2D(16, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 512x512
    x = layers.Conv2D(8, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.UpSampling2D()(x)  # Upsampling to 1024x1024
    x = layers.Conv2D(4, kernel_size=3, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    # Final output layer
    img = layers.Conv2D(3, kernel_size=3, padding='same', activation='tanh')(x)

    # Create and return the model
    model = tf.keras.Model(inputs=style_input, outputs=img)
    return model

# Define the Discriminator (Critic for WGAN)
def build_discriminator():
    input_img = layers.Input(shape=img_shape)

    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(input_img)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)  # No sigmoid activation for the critic

    model = tf.keras.Model(inputs=input_img, outputs=x)
    return model

# Instantiate the models
generator = build_generator()
discriminator = build_discriminator()

# Compile the discriminator model
discriminator.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005))

# Build the WGAN model (combine generator and critic)
discriminator.trainable = False  # During GAN training, freeze the discriminator

z = layers.Input(shape=(latent_dim,))
gen_img = generator(z)
validity = discriminator(gen_img)

wgan = tf.keras.Model(inputs=z, outputs=validity)

# Compile the WGAN model (generator)
wgan.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005))

# Function to train the WGAN with Wasserstein Loss
def train_wgan(epochs, batch_size, sample_interval=50):
    # Load dataset (e.g., CIFAR-10, here we use random data for illustration)
    (X_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()
    X_train = X_train / 127.5 - 1.0  # Rescale to [-1, 1]
    X_train = np.expand_dims(X_train, axis=3)  # Add channel dimension

    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train the Critic (Discriminator)

        # Train the critic 5 times per generator update
        for _ in range(5):  # Number of critic updates
            # Select a random batch of real images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_images = X_train[idx]

            # Generate fake images
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_images = generator.predict(noise)

            # Train the critic on real and fake images (Wasserstein loss)
            c_loss_real = discriminator.train_on_batch(real_images, real_labels)
            c_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
            c_loss = 0.5 * np.add(c_loss_real, c_loss_fake)

            # Clip the critic's weights to enforce the Lipschitz constraint
            for layer in discriminator.layers:
                if hasattr(layer, 'kernel'):
                    layer.kernel.assign(tf.clip_by_value(layer.kernel, -0.01, 0.01))

        # Train the Generator (via the WGAN model)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = wgan.train_on_batch(noise, real_labels)

        # Print the progress
        if epoch % sample_interval == 0:
            print(f"{epoch} [C loss: {c_loss}] [G loss: {g_loss}]")
            sample_images(epoch)

# Function to save generated images
# Function to train the WGAN with Wasserstein Loss
def train_wgan(epochs, batch_size, sample_interval=50):
    # Load dataset (e.g., CIFAR-10, here we use random data for illustration)
    (X_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()

    # Rescale and resize the images to the expected shape
    # array_to_img converts the numpy array back to a PIL image, before using resize
    # img_to_array converts the PIL image to a numpy array
    from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
    X_train_resized = np.array([img_to_array(array_to_img(img).resize((1024, 1024))) for img in X_train])

    X_train_resized = X_train_resized / 127.5 - 1.0  # Rescale to [-1, 1]


    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train the Critic (Discriminator)

        # Train the critic 5 times per generator update
        for _ in range(5):  # Number of critic updates
            # Select a random batch of real images
            idx = np.random.randint(0, X_train_resized.shape[0], batch_size)
            real_images = X_train_resized[idx]

            # Generate fake images
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_images = generator.predict(noise)

            # Train the critic on real and fake images (Wasserstein loss)
            c_loss_real = discriminator.train_on_batch(real_images, real_labels)
            c_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
            c_loss = 0.5 * np.add(c_loss_real, c_loss_fake)

            # Clip the critic's weights to enforce the Lipschitz constraint
            for layer in discriminator.layers:
                if hasattr(layer, 'kernel'):
                    layer.kernel.assign(tf.clip_by_value(layer.kernel, -0.01, 0.01))

        # Train the Generator (via the WGAN model)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = wgan.train_on_batch(noise, real_labels)

        # Print the progress
        if epoch % sample_interval == 0:
            print(f"{epoch} [C loss")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Que.8.Write a function to modify the discriminator to include a dropout layer with a rate of 0.4 and print the
configurations?

In [6]:
def build_discriminator_with_dropout():
    input_img = layers.Input(shape=(32, 32, 3))  # Input shape for CIFAR-10 (32x32x3)

    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(input_img)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    # Add Dropout layer after the last convolutional block
    x = layers.Dropout(0.4)(x)  # Dropout with rate 0.4

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)  # Output layer with no activation for WGAN

    # Build the model
    model = tf.keras.Model(inputs=input_img, outputs=x)

    # Print the model configuration
    model.summary()

    return model


Que.9.Write a function to modify the discriminator to include a dropout layer with a rate of 0.4 and print the
configurations?

In [7]:
import tensorflow as tf
from tensorflow.keras import layers

def build_discriminator_with_dropout():
    input_img = layers.Input(shape=(1024, 1024, 3))  # Input shape for high-resolution images (e.g., 1024x1024x3)

    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(input_img)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    # Add Dropout layer with a rate of 0.4
    x = layers.Dropout(0.4)(x)  # Dropout with a 40% drop rate

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)  # Output layer, no activation function for WGAN

    # Create the model
    model = tf.keras.Model(inputs=input_img, outputs=x)

    # Print the model configuration
    model.summary()  # Prints the layer details, output shapes, and parameters

    return model
